In [1]:
from torchvision.datasets import CIFAR10
from torchvision import transforms,datasets, models
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from PIL import Image
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
trainset = CIFAR10(root='./data', train=True, download=True)
testset = CIFAR10(root='./data', train=False, download=True)

100%|██████████| 170498071/170498071 [00:13<00:00, 12860488.38it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
normalize = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))

transform = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
        
    ])
classDict = {'plane': 0, 'car': 1, 'bird': 2, 'cat': 3, 'deer': 4,
             'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}

In [4]:
class CifarDataset(torch.utils.data.Dataset):

    def __init__(self,dataset,idx,transforms=None,exclude_class = 'frog'):
        self.idx = idx
        self.transforms = transforms
        self.data = torch.utils.data.Subset(dataset, idx)
        self.exclude_class = exclude_class

    def __getitem__(self,index):
        image,label = self.data[index]
        #if(label > classDict[self.exclude_class]):
          #label = label-1
        if(self.transforms!= None):
            image = self.transforms(image)

        return image,label

    def __len__(self):
        return len(self.data)

In [5]:
x_train = trainset.data
x_test = testset.data
y_train = np.array(trainset.targets)
y_test = np.array(testset.targets)

In [6]:
'''train_frogs_idx = np.where(y_train == classDict['frog'])[0]
test_frogs_idx = np.where(y_test == classDict['frog'])[0]

x_train_exclude,x_train_frogs = np.delete(x_train,train_frogs_idx,axis = 0),x_train[train_frogs_idx]
x_test_exclude,x_test_frogs = np.delete(x_train,test_frogs_idx,axis = 0),x_test[test_frogs_idx]'''


BATCH_SIZE = 256
train_dataset = CifarDataset(trainset, np.where(y_train != classDict['frog'])[0], transforms=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,num_workers=2)

In [7]:
#label = [y for x,y in train_dataset]

In [8]:
#set(label)

In [9]:
def train(model,train_dataloader,criterion,optimizer,num_epochs):
  for epoch in range(num_epochs): 
      print(f"Epoch {epoch+1} running") 
      """ Training Phase """
      model.train() 
      running_loss = 0
      running_corrects = 0 
      total = 0
      for i, (inputs, labels) in enumerate(train_dataloader):
          inputs = inputs.to(device)
          labels = labels.to(device) 
          optimizer.zero_grad()
          outputs = model(inputs)
          _, preds = torch.max(outputs, 1)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()
          running_loss += loss.item() * inputs.size(0)
          running_corrects += torch.sum(preds == labels.data)
          total += len(labels)
      epoch_loss = running_loss / total
      epoch_acc = running_corrects / total * 100.
      print('[Train #{}] Loss: {:.4f} Acc: {:.4f}%'.format(epoch+1, epoch_loss, epoch_acc))

In [10]:
def test(model,test_dataloader,criterion,optimizer):
    model.eval()
    with torch.no_grad():
        running_loss = 0.
        running_corrects = 0
        total = 0
        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total += len(labels)
        epoch_loss = running_loss / total
        epoch_acc = running_corrects / total * 100.
        print('[Test] Loss: {:.4f} Acc: {:.4f}% '.format(epoch_loss, epoch_acc))

In [11]:
def get_softmax_values(model,test_dataloader):
    model.eval()
    with torch.no_grad():
        soft_max = []
        total_outliers = 0
        total_samples = 0
        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            softmax_values = torch.nn.functional.softmax(outputs, dim=1)
            for i in range(len(inputs)):
              soft_max.append(np.max(softmax_values[i].cpu().detach().numpy()))
    return np.array(soft_max)
             
                

In [12]:
model = models.resnet18(pretrained = True)   

#num_features = model.fc.in_features    
#model.fc = nn.Linear(num_features, 9) 
model = model.to(device) 
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters())

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 174MB/s]


# Train

In [13]:
train(model,train_loader,criterion,optimizer,200)

Epoch 1 running
[Train #1] Loss: 1.2973 Acc: 60.0756%
Epoch 2 running
[Train #2] Loss: 0.7003 Acc: 75.6600%
Epoch 3 running
[Train #3] Loss: 0.6028 Acc: 78.9911%
Epoch 4 running
[Train #4] Loss: 0.5415 Acc: 81.0733%
Epoch 5 running
[Train #5] Loss: 0.5034 Acc: 82.5822%
Epoch 6 running
[Train #6] Loss: 0.4699 Acc: 83.5867%
Epoch 7 running
[Train #7] Loss: 0.4464 Acc: 84.3800%
Epoch 8 running
[Train #8] Loss: 0.4230 Acc: 85.2133%
Epoch 9 running
[Train #9] Loss: 0.4029 Acc: 85.7444%
Epoch 10 running
[Train #10] Loss: 0.3895 Acc: 86.2289%
Epoch 11 running
[Train #11] Loss: 0.3711 Acc: 86.8844%
Epoch 12 running
[Train #12] Loss: 0.3602 Acc: 87.2356%
Epoch 13 running
[Train #13] Loss: 0.3473 Acc: 87.8844%
Epoch 14 running
[Train #14] Loss: 0.3372 Acc: 88.2222%
Epoch 15 running
[Train #15] Loss: 0.3236 Acc: 88.6200%
Epoch 16 running
[Train #16] Loss: 0.3112 Acc: 89.1489%
Epoch 17 running
[Train #17] Loss: 0.3031 Acc: 89.3044%
Epoch 18 running
[Train #18] Loss: 0.2894 Acc: 89.5822%
Epoch 19 r

In [14]:
torch.save(model.state_dict(),'non_frog.pth')

In [47]:
model = models.resnet18(pretrained = True)   
#num_features = model.fc.in_features    
#model.fc = nn.Linear(num_features, 9) 
model = model.to(device) 
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters())
model.load_state_dict(torch.load('non_frog.pth'))

<All keys matched successfully>

## Test

In [43]:
ttransform = transforms.Compose([
        transforms.ToTensor(),
        normalize
    ])

BATCH_SIZE = 256
non_frog_dataset = CifarDataset(testset, np.where(y_test != classDict['frog'])[0], transforms=ttransform)
non_frog_loader = DataLoader(non_frog_dataset, batch_size=BATCH_SIZE, shuffle=False,num_workers=2)

frog_dataset = CifarDataset(testset, np.where(y_test == classDict['frog'])[0], transforms=ttransform)
frog_loader = DataLoader(frog_dataset, batch_size=BATCH_SIZE, shuffle=False,num_workers=2)

In [48]:
test(model,non_frog_loader,criterion,optimizer)

[Test] Loss: 0.7650 Acc: 86.4444% 


## OOD Detection frog

In [50]:
soft_max = get_softmax_values(model,non_frog_loader)
cutoff = np.percentile(soft_max, 5)
inlier = (soft_max > cutoff).sum() / len(soft_max)
print(f'non frog data => threshold: {cutoff}, inlier percentage: {inlier*100}%')

non frog data => threshold: 0.7182219207286835, inlier percentage: 95.0%


In [51]:
soft_max = get_softmax_values(model,frog_loader)
outlier = (soft_max < cutoff).sum() / len(soft_max)
print(f'frog data => threshold: {cutoff}, outlier percentage: {outlier*100}%')

frog data => threshold: 0.7182219207286835, outlier percentage: 17.2%


## OOD Detection cat

In [52]:
BATCH_SIZE = 256
train_dataset = CifarDataset(trainset, np.where(y_train != classDict['cat'])[0], transforms=transform,exclude_class = 'cat')
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,num_workers=2)

In [34]:
model = models.resnet18(pretrained = True)   
#num_features = model.fc.in_features    
#model.fc = nn.Linear(num_features, 9) 
model = model.to(device) 
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters())

In [35]:
train(model,train_loader,criterion,optimizer,200)

Epoch 1 running
[Train #1] Loss: 1.2607 Acc: 62.3133%
Epoch 2 running
[Train #2] Loss: 0.6166 Acc: 78.9644%
Epoch 3 running
[Train #3] Loss: 0.5162 Acc: 82.3000%
Epoch 4 running
[Train #4] Loss: 0.4689 Acc: 84.0956%
Epoch 5 running
[Train #5] Loss: 0.4272 Acc: 85.3867%
Epoch 6 running
[Train #6] Loss: 0.3969 Acc: 86.3956%
Epoch 7 running
[Train #7] Loss: 0.3749 Acc: 87.0222%
Epoch 8 running
[Train #8] Loss: 0.3489 Acc: 88.1356%
Epoch 9 running
[Train #9] Loss: 0.3352 Acc: 88.5444%
Epoch 10 running
[Train #10] Loss: 0.3167 Acc: 89.2133%
Epoch 11 running
[Train #11] Loss: 0.3057 Acc: 89.5956%
Epoch 12 running
[Train #12] Loss: 0.2933 Acc: 89.9978%
Epoch 13 running
[Train #13] Loss: 0.2829 Acc: 90.3644%
Epoch 14 running
[Train #14] Loss: 0.2773 Acc: 90.4600%
Epoch 15 running
[Train #15] Loss: 0.2597 Acc: 91.0889%
Epoch 16 running
[Train #16] Loss: 0.2462 Acc: 91.3889%
Epoch 17 running
[Train #17] Loss: 0.2425 Acc: 91.6711%
Epoch 18 running
[Train #18] Loss: 0.2352 Acc: 91.9689%
Epoch 19 r

In [36]:
torch.save(model.state_dict(),'non_cat.pth')

In [53]:
model = models.resnet18(pretrained = True)   
#num_features = model.fc.in_features    
#model.fc = nn.Linear(num_features, 9) 
model = model.to(device) 
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters())
model.load_state_dict(torch.load('non_cat.pth'))

<All keys matched successfully>

In [54]:
BATCH_SIZE = 256
non_cat_dataset = CifarDataset(testset, np.where(y_test != classDict['cat'])[0], transforms=ttransform,exclude_class = 'cat')
non_cat_loader = DataLoader(non_cat_dataset, batch_size=BATCH_SIZE, shuffle=False,num_workers=2)

cat_dataset = CifarDataset(testset, np.where(y_test == classDict['cat'])[0], transforms=ttransform,exclude_class = 'cat')
cat_loader = DataLoader(cat_dataset, batch_size=BATCH_SIZE, shuffle=False,num_workers=2)

In [56]:
test(model,non_cat_loader,criterion,optimizer)

[Test] Loss: 0.5636 Acc: 89.6444% 


In [62]:
soft_max = get_softmax_values(model,non_cat_loader)
cutoff = np.percentile(soft_max, 5)

In [63]:
inlier = (soft_max > cutoff).sum() / len(soft_max)
print(f'non cat data => threshold: {cutoff}, inlier percentage: {inlier*100}%')

non cat data => threshold: 0.7593916416168213, inlier percentage: 95.0%


In [64]:
soft_max = get_softmax_values(model,cat_loader)
outlier = (soft_max < cutoff).sum() / len(soft_max)
print(f'cat data => threshold: {cutoff}, outlier percentage: {outlier*100}%')

cat data => threshold: 0.7593916416168213, outlier percentage: 20.200000000000003%
